[View source on GitHub]: https://github.com/wadmp/wadmp.github.io/blob/master/jupyter_notebooks/plot_monitoring_data.ipynb
[Notebook Viewer]: https://nbviewer.jupyter.org/github/wadmp/wadmp.github.io/blob/master/jupyter_notebooks/plot_monitoring_data.ipynb
[Run in binder]: https://mybinder.org/v2/gh/wadmp/wadmp.github.io/master?filepath=jupyter_notebooks%2Fplot_monitoring_data.ipynb

| [![GitHub logo](https://raw.githubusercontent.com/wadmp/wadmp.github.io/master/images/github_logo.png)][View source on GitHub] | [![Jupyter logo](https://raw.githubusercontent.com/wadmp/wadmp.github.io/master/images/jupyter_logo.png)][Notebook Viewer] | [![binder logo](https://raw.githubusercontent.com/wadmp/wadmp.github.io/master/images/binder_logo.png)][Run in binder] |
|:---------------------:|:---------------:|:-------------:|
| [View source on GitHub] | [Notebook Viewer] | [Run in binder] |

## Introduction
This notebook provides an example of using the public REST API of WebAccess/DMP.

It shows how to query the monitoriring data of one device, which is stored in a time-series database called [InfluxDB](https://www.influxdata.com/products/influxdb-overview/).

It uses the [bqplot](https://github.com/bloomberg/bqplot) plotting library.

### Requirements
* If you are running in Jupyter Notebook, you don't need to make any code changes.
* If you are running in Jupyter Lab, you will also need to install these JupyterLab extensions:
  * @jupyter-widgets/jupyterlab-manager
  * bqplot
* You need to have an existing user account on the WA/DMP instance.

### Usage
In the "Global Variables" cell below, change BASE_URL to match the particular WA/DMP instance that you are using.

Then run the cells, either one at a time, or all at once.

When prompted, enter the required User Input (USERNAME, PASSWORD, device_name).

## Setup
This may take a minute ...

In [ ]:
%%capture

# Install packages in the current Jupyter kernel
import sys
!{sys.executable} -m pip install requests

# We have had issues with bqplot due to version incompatibilities.
# To be safe, we pin both the backend (pip) version and the frontend (npm) version.
!{sys.executable} -m pip install bqplot==0.12.6
# bqplot includes ipywidgets, numpy, pandas, so we don't need to install those.

# The following line is commented out because it will cause an exception in Jupyter Notebook.
# HOWEVER, it may be required in Jupyter Lab if you can't install the right version through the Extension Manager.
#!jupyter labextension install bqplot@0.5.6

import requests
import numpy as np
import pandas as pd
import bqplot
import json

## Global variables

In [ ]:
BASE_URL = 'https://gateway.wadmp.com'
BASE_PATH = 'api'
SESSION = requests.Session()  # Use one HTTPS session for all API calls

## User input

In [ ]:
USERNAME = input("Enter username:")
PASSWORD = input("Enter password:")
device_name = input("Enter device alias or MAC address:")

## Login

In [ ]:
url = f"{BASE_URL}/public/auth/connect/token"
credentials = {'username': USERNAME, 'password': PASSWORD, 'client_id': 'python', 'grant_type': 'password'}
print(f"\nSending POST request to {url} with:\n"
        f"    credentials={credentials}")
response = SESSION.post(url, data=credentials)

print(response.status_code)
try:
    print(json.dumps(response.json(), indent=4, sort_keys=True))
except ValueError:
    print(response.text)

if response.status_code == requests.codes['ok']:
    user_token = response.json()["access_token"]
else:
    print("Failed to login!")
    sys.exit(1)

## Use the token in all subsequent API calls

In [ ]:
SESSION.headers.update({'Authorization': f'Bearer {user_token}'})

## Get the device details

In [ ]:
url = f"{BASE_URL}/{BASE_PATH}/management/devices"
query = {'page': 1, 'pageSize': '10', 'name': device_name}
print(f"\nSending GET request to {url} with:\n"
        f"    page=1\n"
        f"    pageSize=10\n"
        f"    name={device_name}\n")
response = SESSION.get(url, params=query)

print(response.status_code)
try:
    print(json.dumps(response.json(), indent=4, sort_keys=True))
except ValueError:
    print(response.text)

if response.status_code == requests.codes['ok']:
    mac = response.json()['data'][0]['mac_address']
    company = response.json()['data'][0]['company']['name']
else:
    print("GET devices query failed!")
    sys.exit(1)

## Get the latest data from InfluxDB
Note that the company name must be included in the WHERE clause!
(If you do not specify a company, the query will default to use the InfluxDB database for your primary company).

And because of the way Grafana [uses variables in queries](https://grafana.com/docs/grafana/latest/features/datasources/influxdb/#using-variables-in-queries), you have to wrap the company name as follows:
`"companyName" =~ /^My Company Inc.$/`

Following the Influx recommendations [here](https://docs.influxdata.com/influxdb/v1.7/troubleshooting/frequently-asked-questions/#when-should-i-single-quote-and-when-should-i-double-quote-in-queries), we single-quote string values and double-quote identifiers.

In [ ]:
url = f"{BASE_URL}/{BASE_PATH}/monitoring/devices/query"
influx_query = f'SELECT "statusTemperature" FROM "SNMP" WHERE ("macAddress" = \'{mac}\' AND "companyName" =~ /^{company}$/ AND time >= now() - 1d )'
query = {'Q': influx_query, 'Epoch': 'ms'}
print(f"\nSending GET request to {url} with:\n"
        f"    Q={influx_query}\n"
        f"    Epoch=ms\n")
response = SESSION.get(url, params=query)

print(response.status_code)
try:
    print(json.dumps(response.json(), indent=4, sort_keys=True))
except ValueError:
    print(response.text)

if response.status_code == requests.codes['ok']:
    # We return a numpy array, because that makes slicing easy
    array = np.array(response.json()['results'][0]['series'][0]['values'])
else:
    print("InfluxDB query failed!")
    sys.exit(1)

## Convert the numpy array to a Pandas Series object

In [ ]:
x = pd.to_datetime(array[:,0], unit='ms')
y = array[:,1]
series = pd.Series(data=y, index=x)
series

## Plot

In [ ]:
# bqplot provides a high-level API called pyplot, which is intended to be similar to matplotlib.
# We use the lower-level "object API", as it provides more flexibility.

x_sc = bqplot.DateScale()
y_sc = bqplot.LinearScale()

ax_x = bqplot.Axis(label='Time', scale=x_sc, grid_lines='solid')
ax_y = bqplot.Axis(label='Temp', scale=y_sc, orientation='vertical', grid_lines='solid')

line = bqplot.Lines(x=x, y=y, scales={'x': x_sc, 'y': y_sc}, marker='circle')

fig = bqplot.Figure(axes=[ax_x, ax_y], marks=[line], title='Temperature')
fig

Note that the X-axis automatically uses the browser's local timezone.
All monitoring data in WebAccess/DMP is stored in UTC, so if your local time is NOT UTC you will notice an offset.